### 抓包代码1.0
#### 一、目标选择
选择移动端的微博（网页版）进行爬取

链接参考来源：<a>https://www.zhihu.com/question/29666539/answer/284798743?utm_id=0</a>

#### 二、分析目标网站
了解目标网站的结构和数据传输方式
![](./URL_weibo.jpg)
这里我们可以发现`URL`的`request`网页：
`https://m.weibo.cn/api/container/getIndex?t=0&luicode=10000011&lfid=100103type%3D1%26q%3DTetraCalyx&type=uid&value=2187177964&containerid=1005052187177964&page=1`
在此`page=1`控制了滚动的页码数，这里提到可以构造页面链接来爬取用户的微博数据
![](./request_head.jpg)
我们得到了另一个相关信息：`Content-Type：application/json; charset=utf-8` 标明了页面数据是通过json返回的，编码类型为utf-8。

另外我们需要注意到另一点：`getindex?=...`类似的js包有两段
![](./about_index.jpg)
第一段为用户卡片之类（关注、签名啥的）
第二段为我们需要爬取的专门用户信息【针对一页的内容来说】

我们需要根据评论的js包得到相关信息：
![](./js_pkg_analyse.png)
其中的id 为构造某一条微博评论链接的关键数据，如果想爬具体某条微博下的评论数据需要用id的值来构造评论页链接。正常可感知的数据：微博内容、评论数、点赞数、分享数、发布时间和来源都是可以直接获取的。

#### 三、模拟浏览器请求
正常爬取微博时需要先登录，这里可以使用post请求将账号和密码传入的方式实现。也可以直接在浏览器内登录后使用登录后的api链接构造请求头，然后使用get方式请求即可

本次按照第二种方式实现。在模拟请求头时，只需构造'User-Agent'和'Referer'这两组信息就行。
```
header=({'User-Agent': '在你的浏览器开发模式下获取此信息',
         'Referer': '在你的浏览器开发模式下获取此信息',
        })
```
这两组信息在页面下方部分获得
![](./info_header.jpg)
构造完请求头，使用requests库进行模拟请求，请求成功网站服务器识别到模拟的浏览器请求会返回网站的json数据，然后通过python的json库来解析它。

In [21]:
# 目前为止的代码
#  _*_ coding: utf-8_*_ # 声明程序的编写字符

import requests as req
import json

pages=9
header=({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.82',
         'Referer': 'https://m.weibo.cn/u/2187177964?t=0&luicode=10000011&lfid=100103type%3D1%26q%3DTetraCalyx'
        })
url='https://m.weibo.cn/api/container/getIndex?t=0&luicode=10000011&lfid=100103type%3D1%26q%3DTetraCalyx&type=uid&value=2187177964&containerid=1076032187177964&%page='

# 构造获取数据的python类
def get_data(header,url,pages):
    data={}
    for i in range(0,pages): # 通过i进行递推构造获取的链接代码
        url=url+str(i)
        data=req.get(url,headers=header,timeout=10) #通过requests的get方法请求
        data=json.loads(data.text) # 解析数据
    return data

data=get_data(header,url,pages)
# print(data)

{'ok': 1, 'data': {'cardlistInfo': {'containerid': '1076032187177964', 'v_p': 42, 'show_style': 1, 'total': 1507, 'autoLoadMoreIndex': 10, 'since_id': 4706088554859139}, 'cards': [{'card_type': 9, 'profile_type_id': 'proweibo_4747033010767561', 'itemid': '1076032187177964_-_4747033010767561', 'scheme': 'https://m.weibo.cn/status/LjJxH3dG1?mblogid=LjJxH3dG1&luicode=10000011&lfid=1076032187177964', 'mblog': {'visible': {'type': 0, 'list_id': 0}, 'created_at': 'Mon Mar 14 21:25:28 +0800 2022', 'id': '4747033010767561', 'mid': '4747033010767561', 'can_edit': False, 'show_additional_indication': 0, 'text': '汪～！', 'source': '微博手机版', 'favorited': False, 'pic_ids': [], 'is_paid': False, 'mblog_vip_type': 0, 'user': {'id': 2187177964, 'screen_name': 'TetraCalyx', 'profile_image_url': 'https://tvax3.sinaimg.cn/crop.0.0.260.260.180/825dafecly8fzddr5uqokj2078078q35.jpg?KID=imgbed,tva&Expires=1689695148&ssig=P%2FwOj2hHmM', 'profile_url': 'https://m.weibo.cn/u/2187177964?luicode=10000011&lfid=107603

#### 四、数据清洗
json数据拿到后，需要对数据进行清洗，筛选出想要的字段信息。

再次分析json返回的数据，所有的微博内容都放在字典data下的['cards']中，按层级关系取出来就可以了。

In [22]:
for i in range(0,pages):
    text=data['data']['cards'][i]['mblog']['text'].strip()
    print(text)

汪～！
//<a href='/n/未見釵頭鳳'>@未見釵頭鳳</a>:北京晚报<span class="url-icon"><img alt=[赞] src="https://h5.sinaimg.cn/m/emoticon/icon/others/h_zan-44ddc70637.png" style="width:1em; height:1em;" /></span>美国欠的不止是解释，是血债
冲啊！！！ //<a href='/n/云呆的哭'>@云呆的哭</a>:冲！！！！//<a href='/n/-森谷moriya-'>@-森谷moriya-</a>:给钱！！！//<a href='/n/江南大野花'>@江南大野花</a>:这感染力！！//<a href='/n/糕不可盼'>@糕不可盼</a>:冲啊——//<a href='/n/免提云层驱动器'>@免提云层驱动器</a>:呜呜呜这张好动人//<a href='/n/荷兰豆本豆豆荷'>@荷兰豆本豆豆荷</a>:蓬勃的生命力！！！//<a href='/n/酒酿熊子'>@酒酿熊子</a>:啊啊啊啊我好爱这张图！！！冲破屏幕的快乐！！！
超顽强！ //<a href='/n/-森中人'>@-森中人</a>:女足太棒了！！！！
//<a href='/n/檀烧'>@檀烧</a>:天哪太棒了！这才值得全国人民大力支持！//<a href='/n/林斜阳笑执清茶'>@林斜阳笑执清茶</a>:太棒了！！！<span class="url-icon"><img alt=[打call] src="https://h5.sinaimg.cn/m/emoticon/icon/default/fb_a1dacall-1e0c4593fc.png" style="width:1em; height:1em;" /></span><span class="url-icon"><img alt=[打call] src="https://h5.sinaimg.cn/m/emoticon/icon/default/fb_a1dacall-1e0c4593fc.png" style="width:1em; height:1em;" /></span><span class="url-icon"><img alt=[打call] src="

### 抓包代码2.0
#### 以“张天爱”微博为例：
1. 找到目标信息：
+ cookie：
![](./2_cookie.jpg)
```
ALF=1692107211; SUB=_2A25JsvAqDeRhGeBO7lcW9S3Iwj-IHXVrXJBirDV6PUJbkdCOLUrWkW1NRf6whRBDNroUOM2t-BHLW6ZYM07gqWhP; _T_WM=45099081067; WEIBOCN_FROM=1110006030; MLOGIN=1; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D100103type%253D1%2526q%253D%25E5%25BC%25A0%25E5%25A4%25A9%25E7%2588%25B1%26uicode%3D10000011%26fid%3D1076031788283193; XSRF-TOKEN=4f0365; mweibo_short_token=913ebaeb2b
```
2. 相关代码：
+ 导入相关需要的库：

In [2]:
import os
import re # 正则表达式提取文本
from jsonpath import jsonpath # 解析json数据
import requests # 发送请求
import pandas as pd # 存取csv文件
import datetime

+ 然后，定义一个转换时间字符串的函数，因为爬取到的时间戳是GMT格式（类似这种：Fri Jun 17 22:21:48 +0800 2022）的，需要转换成标准格式：

In [3]:
# 将新浪微博的时间转换作标准格式
def trans_time(v_str):
    # 转换GMT时间为标准格式
    GMT_FORMAT='%a %b %d %H:%M:%S +0800 %Y'
    timeArray=datetime.datetime.strptime(v_str,GMT_FORMAT)
    ret_time=timeArray.strftime('%Y-%m-%d %H:%M:%S')
    return ret_time

+ 定义一个请求头，后面发送请求的时候带上它，防止反爬

In [4]:
# 请求头
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Mobile Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}

+ 打开chrome浏览器，按F12进入开发者模式，按照下图操作，分析出目标请求地址：
![](./2_fetch.jpg)
根据分析结果，编写请求代码：

In [7]:
# 相关代码：
if page==1: # 第一页，没有max_id参数
    url='https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id_type=0'.format(weibo_id,weibo_id)
else: # 非第一页，需要max_id参数
    if max_id=='0': # 如果max_id==0，则没有下一页，break结束循环
        break
        print('max_id is zero, break now!')
    url='https://m.weibo.cn/comments/hotflow?id={}&mid={}&max_id_type={}'.format(weibo_id,weibo_id,max_id)

如果是第一页，不用传max_id参数。
如果非第一页，需要传max_id参数，它的值来自于上一页的`r.json()['data']['max_id']`
下面，就是正常爬虫逻辑了。
首先，向微博页面发送请求：

In [ ]:
r=requests.get(url,headers=headers) # 发送请求
printi(r.status_codea) #查看响应码
print(r.json()) #查看响应内容

下面是解析数据的处理逻辑：

In [ ]:
datas = r.json()['data']['data']
for data in datas:
    page_list.append(page)
    id_list.append(data['id'])
    dr = re.compile(r'<[^>]+>', re.S)  # 用正则表达式清洗评论数据
    text2 = dr.sub('', data['text'])
    text_list.append(text2)  # 评论内容
    time_list.append(trans_time(v_str=data['created_at']))  # 评论时间
    like_count_list.append(data['like_count'])  # 评论点赞数
    source_list.append(data['source'])  # 评论者IP归属地
    user_name_list.append(data['user']['screen_name'])  # 评论者姓名
    user_id_list.append(data['user']['id'])  # 评论者id
    user_gender_list.append(tran_gender(data['user']['gender']))  # 评论者性别
    follow_count_list.append(data['user']['follow_count'])  # 评论者关注数
    followers_count_list.append(data['user']['followers_count'])  # 评论者粉丝数

最后，是保存数据的处理逻辑：

In [ ]:
df = pd.DataFrame(
    {
        '微博id': [weibo_id] * len(time_list),
        '评论页码': page_list,
        '评论id': id_list,
        '评论时间': time_list,
        '评论点赞数': like_count_list,
        '评论者IP归属地': source_list,
        '评论者姓名': user_name_list,
        '评论者id': user_id_list,
        '评论者性别': user_gender_list,
        '评论者关注数': follow_count_list,
        '评论者粉丝数': followers_count_list,
        '评论内容': text_list,
    }
)
if os.path.exists(v_comment_file):  # 如果文件存在，不再设置表头
    header = False
else:  # 否则，设置csv文件表头
    header = True
# 保存csv文件
df.to_csv(v_comment_file, mode='a+', index=False, header=header, encoding='utf_8_sig')
print('结果保存成功:{}'.format(v_comment_file))

篇幅有限，请求头、cookie、循环页码、数据清洗等其他细节不再赘述。